In [23]:
%load_ext autoreload
%autoreload 2

In [24]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import gcf

import os
import numpy as np
import pandas as pd 
import sys
import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import VoxelModelCache
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures
from mcmodels.core.connectivity_data import get_connectivity_data
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique #, unionize
from mcmodels.core.utils import get_leaves_ontologicalorder, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3,get_indices_2ormore
def get_aligned_ids(list1,list2):
    output = np.empty(len(list1), dtype = int)
    for i in range(len(list1)):
        output[ i] = np.intersect1d(st.ancestor_ids([list1[i]]), list2)[0]
    return(output)

def fix_pdcsv(csv):
    
    #wt_conn = pd.read_csv('/Users/samsonkoelle/wt_leafsmooth_leafsurface_sumtarget.csv',  header=[0,1])
    csv_rownames = np.asarray(csv.iloc[:,0])
    csv = csv.iloc[:,1:]
    csv.index = csv_rownames
    return(csv)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
#read data
TOP_DIR = workingdirectory
INPUT_JSON = workingdirectory + '/data/meta/input_011520.json'
EXPERIMENTS_EXCLUDE_JSON = workingdirectory + '/data/meta/experiments_exclude.json'

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#NOTE (Sam): hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel(workingdirectory + '/data/data_final/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)

#NOTE (Sam): switched from v2 structures to v3
#ontological_order = get_ordered_summary_structures(cache)
ontological_order = get_ordered_summary_structures(cache,167587189)

#NOTE (Sam): due to redundancy in ccfv3, its necessary to remove 'MDRNv', 'MDRNd' from summary structures (include as leafs of MDRN)
#we retain ordering
ontological_order = ontological_order[~np.in1d(ontological_order,[1098, 1107])]

#NOTE (Sam): this was (seemingly mistakenly) mouse_connectivity_manifest.json in an earlier version
mcc = MouseConnectivityCache(manifest_file = workingdirectory + '/data/meta/new_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]

annotation/ccf_2017


In [38]:
#NOTE (Sam): this will download the data if necessary 
connectivity_data = get_connectivity_data(cache, major_structure_ids, experiments_exclude, remove_injection = False, structure_set_id=167587189)
connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)
connectivity_data.ai_map = ai_map
connectivity_data.get_summarystructures(data_info)

512
703
1089


2021-09-13 15:19:01,561 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/642967852?image=data_mask&resolution=100
2021-09-13 15:19:01,697 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/642967852?image=injection_density&resolution=100
2021-09-13 15:19:01,787 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/642967852?image=injection_fraction&resolution=100
2021-09-13 15:19:01,868 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/642967852?image=projection_density&resolution=100
2021-09-13 15:19:02,154 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/293011759?image=data_mask&resolution=100
2021-09-13 15:19:02,283 allensdk.api.api.retrieve_file_over_http INFO

2021-09-13 15:19:06,350 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/182295393?image=data_mask&resolution=100
2021-09-13 15:19:06,480 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/182295393?image=injection_density&resolution=100
2021-09-13 15:19:06,581 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/182295393?image=injection_fraction&resolution=100
2021-09-13 15:19:06,675 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/182295393?image=projection_density&resolution=100
2021-09-13 15:19:06,813 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/302217570?image=data_mask&resolution=100
2021-09-13 15:19:06,915 allensdk.api.api.retrieve_file_over_http INFO

2021-09-13 15:19:10,898 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/301421960?image=data_mask&resolution=100
2021-09-13 15:19:10,988 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/301421960?image=injection_density&resolution=100
2021-09-13 15:19:11,071 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/301421960?image=injection_fraction&resolution=100
2021-09-13 15:19:11,152 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/301421960?image=projection_density&resolution=100
2021-09-13 15:19:11,278 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/178382220?image=data_mask&resolution=100
2021-09-13 15:19:11,372 allensdk.api.api.retrieve_file_over_http INFO

2021-09-13 15:19:15,503 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/180297139?image=data_mask&resolution=100
2021-09-13 15:19:15,598 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/180297139?image=injection_density&resolution=100
2021-09-13 15:19:15,686 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/180297139?image=injection_fraction&resolution=100
2021-09-13 15:19:15,778 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/180297139?image=projection_density&resolution=100
2021-09-13 15:19:15,921 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/113935285?image=data_mask&resolution=100
2021-09-13 15:19:16,013 allensdk.api.api.retrieve_file_over_http INFO

...

2021-09-13 15:30:37,625 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/286553311?image=injection_fraction&resolution=100
2021-09-13 15:30:37,700 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/286553311?image=projection_density&resolution=100
2021-09-13 15:30:37,799 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/178489574?image=data_mask&resolution=100
2021-09-13 15:30:37,890 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/178489574?image=injection_density&resolution=100
2021-09-13 15:30:37,973 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/178489574?image=injection_fraction&resolution=100
2021-09-13 15:30:38,058 allensdk.api.api.retrieve_file_over_

2021-09-13 15:30:42,067 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/299732738?image=injection_fraction&resolution=100
2021-09-13 15:30:42,153 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/299732738?image=projection_density&resolution=100
2021-09-13 15:30:42,296 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/166267651?image=data_mask&resolution=100
2021-09-13 15:30:42,392 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/166267651?image=injection_density&resolution=100
2021-09-13 15:30:42,474 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/166267651?image=injection_fraction&resolution=100
2021-09-13 15:30:42,560 allensdk.api.api.retrieve_file_over_

2021-09-13 15:30:46,968 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/180520257?image=injection_fraction&resolution=100
2021-09-13 15:30:47,080 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/180520257?image=projection_density&resolution=100
2021-09-13 15:30:47,192 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/168095041?image=data_mask&resolution=100
2021-09-13 15:30:47,302 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/168095041?image=injection_density&resolution=100
2021-09-13 15:30:47,375 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/168095041?image=injection_fraction&resolution=100
2021-09-13 15:30:47,457 allensdk.api.api.retrieve_file_over_

2021-09-13 15:30:51,434 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/265287564?image=injection_fraction&resolution=100
2021-09-13 15:30:51,524 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/265287564?image=projection_density&resolution=100
2021-09-13 15:30:51,629 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/479671695?image=data_mask&resolution=100
2021-09-13 15:30:51,748 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/479671695?image=injection_density&resolution=100
2021-09-13 15:30:51,834 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/479671695?image=injection_fraction&resolution=100
2021-09-13 15:30:51,910 allensdk.api.api.retrieve_file_over_

2021-09-13 15:30:55,870 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/113784293?image=injection_fraction&resolution=100
2021-09-13 15:30:55,956 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/113784293?image=projection_density&resolution=100
2021-09-13 15:30:56,081 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/278398949?image=data_mask&resolution=100
2021-09-13 15:30:56,180 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/278398949?image=injection_density&resolution=100
2021-09-13 15:30:56,269 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/278398949?image=injection_fraction&resolution=100
2021-09-13 15:30:56,356 allensdk.api.api.retrieve_file_over_

In [40]:
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)

In [41]:
np.save(workingdirectory + '/data/meta/default_structures', default_structures)
np.save(workingdirectory + '/data/meta/ontological_order_v3', ontological_order)
np.save(workingdirectory + '/data/meta/major_structures',major_structures)
np.save(workingdirectory + '/data/meta/major_structure_ids',major_structure_ids)
np.save(workingdirectory + '/data/meta/default_structure_ids', default_structure_ids)
np.save(workingdirectory + '/data/meta/ontological_order_leaves_v3', ontological_order_leaves)

In [6]:
# wt_conn = pd.read_csv(workingdirectory + '/paper/connectivities/elsumsurfleafsmth_v3_leafsum_wt.csv', header = [0,1])
# wt_conn = fix_pdcsv(wt_conn)
# targ_sum = np.asarray(list(wt_conn.columns))[:,1]
# targ_sum_id = np.asarray([ai_map[targ_sum[i]] for i in range(len(targ_sum))])
# targ_sum_major = get_aligned_ids(targ_sum_id,major_structure_ids)
# source_leaf_major = get_aligned_ids(ontological_order_leaves,major_structure_ids)
# source_leaf_major_names = np.asarray([ia_map[source_leaf_major[i]] for i in range(len(source_leaf_major))])
# targ_sum_major_names = np.asarray([ia_map[targ_sum_major[i]] for i in range(len(targ_sum_major))])
# np.save(workingdirectory + '/paper/info/source_leaf_major_names', source_leaf_major_names)
# np.save(workingdirectory + '/paper/info/targ_sum_major_names', targ_sum_major_names)